Michael Felzan

Arc II

Lab 4 Notebook

# 1.) GENERAL INITIALIZATION:

## A.) Importing Packages:

In [78]:
import arcpy
from arcpy.sa import *
import requests
import io
from io import StringIO
import datetime
import zipfile
import os
import pandas as pd
import csv
import glob
import shutil

### B.) Manually inputting the path to the base of script users working directory:

In [4]:
repo_base = r'C:\\Users\\michaelfelzan\\Documents\\arc ii lab IV' # input the repo you want to
                                                                  # 'house' the NDAWN folder structure

### C.) Grabbing current date from the datetime module; creating a new folder for today's date:

In [7]:
daterightnow = datetime.datetime.now()
formatteddate = daterightnow.strftime("%Y-%m-%d")  

currentdaywkdirpath = os.path.join(repo_base, formatteddate + "_NDAWN")
os.mkdir(currentdaywkdirpath)
wkdir = currentdaywkdirpath

In [8]:
wkdir

'C:\\\\Users\\\\michaelfelzan\\\\Documents\\\\arc ii lab IV\\2021-04-21_NDAWN'

# 2.) DIRECTORY/ STATION DICTIONARY INITIALIZATION:

## D.) Setting up dictionaries / lists to house all relevant info about every station

#### (I found this URL by clicking somewhere on the NDAWN website in Lab 1; the URL contains every station number/ID:

In [10]:
all_stations_URL = "https://ndawn.ndsu.nodak.edu/get-table.html?station=78&station=111&station=98&station=142&station=138&station=9&station=10&station=118&station=56&station=11&station=12&station=58&station=13&station=84&station=55&station=7&station=87&station=14&station=15&station=96&station=16&station=137&station=124&station=17&station=85&station=140&station=134&station=18&station=136&station=65&station=104&station=99&station=19&station=129&station=20&station=101&station=81&station=21&station=97&station=22&station=75&station=2&station=139&station=23&station=62&station=86&station=24&station=89&station=126&station=93&station=90&station=25&station=83&station=107&station=77&station=26&station=70&station=127&station=27&station=132&station=28&station=29&station=30&station=31&station=102&station=32&station=119&station=4&station=80&station=33&station=59&station=105&station=82&station=34&station=72&station=135&station=35&station=76&station=120&station=141&station=109&station=36&station=79&station=71&station=37&station=38&station=39&station=130&station=73&station=40&station=41&station=54&station=69&station=113&station=128&station=42&station=43&station=103&station=116&station=88&station=114&station=3&station=64&station=115&station=67&station=44&station=133&station=106&station=100&station=121&station=45&station=46&station=61&station=66&station=74&station=60&station=125&station=8&station=47&station=122&station=108&station=5&station=48&station=68&station=49&station=50&station=91&station=117&station=63&station=51&station=6&station=52&station=92&station=112&station=131&station=123&station=95&station=53&station=57&station=110&variable=mdsr&dfn=&dfy=&year=2020&ttype=monthly&quick_pick=&begin_date=2020-02&count=12"
all_stations_URL

'https://ndawn.ndsu.nodak.edu/get-table.html?station=78&station=111&station=98&station=142&station=138&station=9&station=10&station=118&station=56&station=11&station=12&station=58&station=13&station=84&station=55&station=7&station=87&station=14&station=15&station=96&station=16&station=137&station=124&station=17&station=85&station=140&station=134&station=18&station=136&station=65&station=104&station=99&station=19&station=129&station=20&station=101&station=81&station=21&station=97&station=22&station=75&station=2&station=139&station=23&station=62&station=86&station=24&station=89&station=126&station=93&station=90&station=25&station=83&station=107&station=77&station=26&station=70&station=127&station=27&station=132&station=28&station=29&station=30&station=31&station=102&station=32&station=119&station=4&station=80&station=33&station=59&station=105&station=82&station=34&station=72&station=135&station=35&station=76&station=120&station=141&station=109&station=36&station=79&station=71&station=37&

##### Extracting the station numbers from the URL:

In [11]:
frontslice = all_stations_URL.split('get-table.html?')[1]
stationslice = frontslice.split('&variable=mdsr')[0]
split_stations = stationslice.split("&")

split_stations[0:8]

['station=78',
 'station=111',
 'station=98',
 'station=142',
 'station=138',
 'station=9',
 'station=10',
 'station=118']

##### Creating list of *just* the station numbers:

In [12]:
just_stationnumbers = []

for stationequals in split_stations:
    splitatequals = stationequals.split("=")
    just_stationnumbers.append(splitatequals[1])

just_stationnumbers[0:13]

['78',
 '111',
 '98',
 '142',
 '138',
 '9',
 '10',
 '118',
 '56',
 '11',
 '12',
 '58',
 '13']

In [90]:
len(just_stationnumbers)

140

### E.) Iteratively sending requests to NDAWN server for each of the station numbers (via inputting the station numbers into the URL parameters) to receive the city/state that corresponds to each station number.

In [13]:
station_info_url_base = "https://ndawn.ndsu.nodak.edu/station-info.html"

In [14]:
def StationNameGetter(splitstation):
    full_url = station_info_url_base+"?"+splitstation
    html_text = (requests.get(full_url)).text
    frontslice = html_text.split("<h1>NDAWN Station: ")[1]
    clean_name = frontslice.split("</h1>")[0]
    return clean_name

StationNameGetter('station=78')

'Ada, MN'

##### Using 'StationNameGetter' function created above to request the city/state names which correspond to each station number:

In [15]:
station_names = {}

for station in split_stations:
    iter_station = station.replace("=", " ")
    iter_name = StationNameGetter(station)
    station_names[iter_station] = iter_name

print(station_names)

{'station 78': 'Ada, MN', 'station 111': 'Adams, ND', 'station 98': 'Alamo, ND', 'station 142': 'Amidon, ND', 'station 138': 'Arnegard, ND', 'station 9': 'Baker, ND', 'station 10': 'Beach, ND', 'station 118': 'Becker, MN', 'station 56': 'Berthold, ND', 'station 11': 'Bismarck, ND', 'station 12': 'Bottineau, ND', 'station 58': 'Bowbells, ND', 'station 13': 'Bowman, ND', 'station 84': 'Brampton, ND', 'station 55': 'Britton, SD', 'station 7': 'Brorson, MT', 'station 87': 'Campbell, MN', 'station 14': 'Cando, ND', 'station 15': 'Carrington, ND', 'station 96': 'Carson, ND', 'station 16': 'Cavalier, ND', 'station 137': 'Charbonneau, ND', 'station 124': 'Clarissa, MN', 'station 17': 'Columbus, ND', 'station 85': 'Cooperstown, ND', 'station 140': 'Courtenay, ND', 'station 134': 'Crane Creek, ND', 'station 18': 'Crary, ND', 'station 136': 'Croff, ND', 'station 65': 'Crosby, ND', 'station 104': 'Crystal, ND', 'station 99': 'Dagmar, MT', 'station 19': 'Dazey, ND', 'station 129': 'Denhoff, ND', 's

In [16]:
station_names["station 102"]

'Hope, ND'

### F.) Writing a CSV file at base of directory which contains all station numbers and their city/state names:

In [28]:
stationnamez = os.path.join(repo_base, "stationnamez.csv")

with open(stationnamez, "w", newline="") as outfile:
    writer = csv.writer(outfile)
    for key, value in (list(station_names.items())):
        writer.writerow([key, value])

### G.) Creating a 'STATIONS' folder at the base of working directory:

In [29]:
stationsfolderpath = os.path.join(wkdir, 'STATIONS')
os.mkdir(stationsfolderpath)

### H.) Iteratively creating folders inside STATIONS folder for each of the stations (named by just their station #):


In [30]:
for stationnum in just_stationnumbers:
    nameoffolder = os.path.join(stationsfolderpath, stationnum)
    os.mkdir(nameoffolder)

# 3.) REQUESTING AND SEQUESTERING AVE. TEMPERATURE DATA FOR THE PAST 30 DAYS FOR EACH WEATHER STATION

### I.) Iterating over list of stations and sending requests to NDAWN to retrieve average temperature data for each station for the past 30 days:

In [31]:
ndawn_csvtable_urlbase = "https://ndawn.ndsu.nodak.edu/table.csv"

In [41]:
for stationnum in just_stationnumbers:
    foldpath = os.path.join(stationsfolderpath, stationnum)
    master_params = {
        "station" : stationnum,
        "variable" : "ddavt",
        "dfn" : "",
        "year" : "2021",      # NDAWN has a built-in option to
        "ttype" : "daily",    # "quickpick" the past 30 days' data
        "quick_pick" : "30_d",        # Past 30 days,
        "begin_date" : formatteddate}  # Based on current date.
    
    reqreq = requests.get(ndawn_csvtable_urlbase,
                          params = master_params)
    
    csvname = os.path.join(foldpath, f'{stationnum}_initial.csv')
    with open(csvname, "w",newline="") as open_csv:
        open_csv.write(reqreq.content.decode('utf-8'))

### J.) Now that we've received CSV tables for each station's ave temp values for the past 30 days, we may clean these CSV files up to have ~only~ a descriptive header and contents....so we may then concatenate all of the data into one CSV:

##### First we will create a new folder at the base of the working directory called 'CleanAveTempCSVs', which will store the 'cleaned up' csv tables for each station's 30 day ave. temp:

In [34]:
cleanavetemppath = os.path.join(wkdir, 'CleanAveTempCSVs')
os.mkdir(cleanavetemppath)

##### Now, running the function which lops off excess rows on each station's 30-day ave temp CSV, and gives the tables clean headers:

In [42]:
for stationnum in just_stationnumbers:
    folderpath = os.path.join(stationsfolderpath,
                              stationnum)
    itercsvname = os.path.join(folderpath,
                               f'{stationnum}_initial.csv')
    with open(itercsvname, 'r',newline="") as itercsv:
        rowitems = []
        for row in itercsv:
            rowitems.append(row)
        if len(rowitems) < 6: # some of the stations do not have data for
                              # the past 30 days; by selecting only CSVs
                              # which have more than 6 rows, we should be
            pass              # selecting all tables which returned data    
        else:
            station_avetemp_df = pd.read_csv(itercsvname,
                                             skiprows=4)
                                        # skipping first four rows,
                                        # so df will only represent
                                        # header and raw data.
            updatedDF = station_avetemp_df.rename(columns={
                "Unnamed: 0": "Station Name",
                "deg" : "Latitude",
                "deg.1" : "Longitude",    # all dfs should have the same wonky
                "ft" : "Elevation",       # variable names.
                "Unnamed: 4" : "Year",
                "Unnamed: 5" : "Month",
                "Unnamed: 6" : "Day",
                "Degrees F" : "Avg Temp",
                "Unnamed: 8" : "Avg Temp Flag",
                "Degrees F.1" : "Normal Avg Temp",
                "Degrees F.2" : "Departure from Normal Avg Temp"})
        
            outcleancsv = os.path.join(wkdir,
                                       'CleanAveTempCSVs',
                                       f'{stationnum}_clean.csv')
            updatedDF.to_csv(outcleancsv)

#### Combining all stations 30-day ave temp CSVs into one 'master' concatenated CSV:

In [48]:
avetemp30dayconcat = os.path.join(cleanavetemppath,
                                  'avetemp30dayconcat.csv')

allFiles = glob.glob(cleanavetemppath + "/*.csv")
allFiles.sort()
with open(avetemp30dayconcat, 'wb') as outfile:
    for i, fname in enumerate(allFiles):
        with open(fname, 'rb') as infile:
            if i != 0:
                infile.readline()  # Throw away header on all but first file
            # Block copy rest of file from input to output without parsing
            shutil.copyfileobj(infile, outfile)
            print(fname + " has been imported.")

C:\\Users\\michaelfelzan\\Documents\\arc ii lab IV\2021-04-21_NDAWN\CleanAveTempCSVs\100_clean.csv has been imported.
C:\\Users\\michaelfelzan\\Documents\\arc ii lab IV\2021-04-21_NDAWN\CleanAveTempCSVs\101_clean.csv has been imported.
C:\\Users\\michaelfelzan\\Documents\\arc ii lab IV\2021-04-21_NDAWN\CleanAveTempCSVs\102_clean.csv has been imported.
C:\\Users\\michaelfelzan\\Documents\\arc ii lab IV\2021-04-21_NDAWN\CleanAveTempCSVs\103_clean.csv has been imported.
C:\\Users\\michaelfelzan\\Documents\\arc ii lab IV\2021-04-21_NDAWN\CleanAveTempCSVs\104_clean.csv has been imported.
C:\\Users\\michaelfelzan\\Documents\\arc ii lab IV\2021-04-21_NDAWN\CleanAveTempCSVs\105_clean.csv has been imported.
C:\\Users\\michaelfelzan\\Documents\\arc ii lab IV\2021-04-21_NDAWN\CleanAveTempCSVs\106_clean.csv has been imported.
C:\\Users\\michaelfelzan\\Documents\\arc ii lab IV\2021-04-21_NDAWN\CleanAveTempCSVs\107_clean.csv has been imported.
C:\\Users\\michaelfelzan\\Documents\\arc ii lab IV\2021-

#### Creating a new folder which will contain CSVs describing each station's monthly mean ave temp:

In [37]:
avetempgbpath = os.path.join(wkdir,
                             'AveTempGroupbyCSVS')
os.mkdir(avetempgbpath)

#### Using Pandas 'GroupBy' function to aggregate all 30 days ave temp data for each station into one mean value; writing new CSVs to 'AveTempGroupbyCSVS' folder:

In [49]:
for stationnum in just_stationnumbers:
    folderpath = os.path.join(stationsfolderpath,
                              stationnum)
    itercsvname = os.path.join(folderpath,
                               f'{stationnum}_initial.csv')
    with open(itercsvname, 'r',newline="") as itercsv:
        rowitems = []
        for row in itercsv:
            rowitems.append(row)
        if len(rowitems) < 6:
            pass
        else:
            station_avetemp_df = pd.read_csv(itercsvname,
                                             skiprows=4)
            updatedDF = station_avetemp_df.rename(columns={
                "Unnamed: 0": "Station Name",
                "deg" : "Latitude",       # only changing header names that
                "deg.1" : "Longitude",    # will be included in the groupby table
                "Degrees F" : "30 Day Avg Temp"})
            outavetempcsv = os.path.join(wkdir,
                                         'AveTempGroupbyCSVS',
                                         f'{stationnum}_avetempgroupby.csv')
            groupbye = updatedDF.groupby(['Station Name','Latitude','Longitude']).agg({'30 Day Avg Temp': ['mean']}).reset_index()
                            # grouping by mean will not affect the lat and long values,
                            # because they are the same for every day
            groupbye.columns = groupbye.columns.get_level_values(0)
            groupbye.to_csv(outavetempcsv)

##### Example of what one 'ave temp groupby' CSV file's contents look like:

In [51]:
groupbye

,Station Name,Latitude,Longitude,30 Day Avg Temp
0,Zeeland,46.013378,-99.687587,36.8081


### K.) Concatenating all stations' Ave Temp Groupby CSVS into one master CSV. This CSV contains each station name, its Latitude, Longitude, and mean 30 Day Ave Temp:

In [52]:
avetempGBconcat = os.path.join(avetempgbpath,
                               'avetempGBconcat.csv')

allFiles = glob.glob(avetempgbpath + "/*.csv")
allFiles.sort()
with open(avetempGBconcat, 'wb') as outfile:
    for i, fname in enumerate(allFiles):
        with open(fname, 'rb') as infile:
            if i != 0:
                infile.readline()  # Throw away header on all but first file
            # Block copy rest of file from input to output without parsing
            shutil.copyfileobj(infile, outfile)
            print(fname + " has been imported.")

C:\\Users\\michaelfelzan\\Documents\\arc ii lab IV\2021-04-21_NDAWN\AveTempGroupbyCSVS\100_avetempgroupby.csv has been imported.
C:\\Users\\michaelfelzan\\Documents\\arc ii lab IV\2021-04-21_NDAWN\AveTempGroupbyCSVS\101_avetempgroupby.csv has been imported.
C:\\Users\\michaelfelzan\\Documents\\arc ii lab IV\2021-04-21_NDAWN\AveTempGroupbyCSVS\102_avetempgroupby.csv has been imported.
C:\\Users\\michaelfelzan\\Documents\\arc ii lab IV\2021-04-21_NDAWN\AveTempGroupbyCSVS\103_avetempgroupby.csv has been imported.
C:\\Users\\michaelfelzan\\Documents\\arc ii lab IV\2021-04-21_NDAWN\AveTempGroupbyCSVS\104_avetempgroupby.csv has been imported.
C:\\Users\\michaelfelzan\\Documents\\arc ii lab IV\2021-04-21_NDAWN\AveTempGroupbyCSVS\105_avetempgroupby.csv has been imported.
C:\\Users\\michaelfelzan\\Documents\\arc ii lab IV\2021-04-21_NDAWN\AveTempGroupbyCSVS\106_avetempgroupby.csv has been imported.
C:\\Users\\michaelfelzan\\Documents\\arc ii lab IV\2021-04-21_NDAWN\AveTempGroupbyCSVS\107_avetem

C:\\Users\\michaelfelzan\\Documents\\arc ii lab IV\2021-04-21_NDAWN\AveTempGroupbyCSVS\62_avetempgroupby.csv has been imported.
C:\\Users\\michaelfelzan\\Documents\\arc ii lab IV\2021-04-21_NDAWN\AveTempGroupbyCSVS\63_avetempgroupby.csv has been imported.
C:\\Users\\michaelfelzan\\Documents\\arc ii lab IV\2021-04-21_NDAWN\AveTempGroupbyCSVS\64_avetempgroupby.csv has been imported.
C:\\Users\\michaelfelzan\\Documents\\arc ii lab IV\2021-04-21_NDAWN\AveTempGroupbyCSVS\65_avetempgroupby.csv has been imported.
C:\\Users\\michaelfelzan\\Documents\\arc ii lab IV\2021-04-21_NDAWN\AveTempGroupbyCSVS\66_avetempgroupby.csv has been imported.
C:\\Users\\michaelfelzan\\Documents\\arc ii lab IV\2021-04-21_NDAWN\AveTempGroupbyCSVS\67_avetempgroupby.csv has been imported.
C:\\Users\\michaelfelzan\\Documents\\arc ii lab IV\2021-04-21_NDAWN\AveTempGroupbyCSVS\68_avetempgroupby.csv has been imported.
C:\\Users\\michaelfelzan\\Documents\\arc ii lab IV\2021-04-21_NDAWN\AveTempGroupbyCSVS\69_avetempgroupby

# 4.) REQUESTING AND SEQUESTERING ABSOLUTE MIN AND MAX TEMPERATURE VALUES OVER THE PAST 30 DAYS FOR EACH WEATHER STATION

#### Assigning variables to the URL params NDAWN uses to specify what temperature variable should be used in the CSV return (max daily temp, min daily temp, ave daily temp, etc.)

In [53]:
maxtempvar = 'ddmxt'
mintempvar = 'ddmnt'

### L.) Creating new directories for the Max Temp and Min Temp station data:

In [54]:
cleanmaxtemp = os.path.join(wkdir, 'CleanMaxTempCSVs')
os.mkdir(cleanmaxtemp)

cleanmintemp = os.path.join(wkdir, 'CleanMinTempCSVs')
os.mkdir(cleanmintemp)

maxtempgroupby = os.path.join(wkdir, 'MaxTempGroupbyCSVs')
os.mkdir(maxtempgroupby)

mintempgroupby = os.path.join(wkdir, 'MinTempGroupbyCSVs')
os.mkdir(mintempgroupby)

### M.) Sending requests to NDAWN for CSVs of daily MAXIMUM temp over 30 days for each of the weather stations:

In [56]:
for stationnum in just_stationnumbers:
    foldpath = os.path.join(stationsfolderpath, stationnum)
    master_params = {
        "station" : stationnum,
        "variable" : maxtempvar,  # max temp
        "dfn" : "",
        "year" : "2021",     
        "ttype" : "daily",    
        "quick_pick" : "30_d",        
        "begin_date" : formatteddate}  
    
    reqreq = requests.get(ndawn_csvtable_urlbase,
                          params = master_params)
    
    csvname = os.path.join(foldpath,
                           f'{stationnum}_maxtemp.csv') 
    with open(csvname, "w",newline="") as open_csv:
        open_csv.write(reqreq.content.decode('utf-8'))

### N.) Sending requests to NDAWN for CSVs of daily MINIMUM temp over 30 days for each of the weather stations:

In [58]:
for stationnum in just_stationnumbers:
    foldpath = os.path.join(stationsfolderpath, stationnum)
    master_params = {
        "station" : stationnum,
        "variable" : mintempvar,  # !! MIN !! temp
        "dfn" : "",
        "year" : "2021",      
        "ttype" : "daily",    
        "quick_pick" : "30_d",        
        "begin_date" : formatteddate}  
    
    reqreq = requests.get(ndawn_csvtable_urlbase,
                          params = master_params)
    
    csvname = os.path.join(foldpath,
                           f'{stationnum}_mintemp.csv')
    with open(csvname, "w",newline="") as open_csv:
        open_csv.write(reqreq.content.decode('utf-8'))

##### Running previous function which lops off excess rows on each station's 30-day max temp CSV, and gives the tables clean headers:

In [59]:
for stationnum in just_stationnumbers:
    folderpath = os.path.join(stationsfolderpath,
                              stationnum)
    itercsvname = os.path.join(folderpath,
                               f'{stationnum}_maxtemp.csv')
    with open(itercsvname, 'r',newline="") as itercsv:
        rowitems = []
        for row in itercsv:
            rowitems.append(row)
        if len(rowitems) < 6:             
            pass   
        else:
            station_maxtemp_df = pd.read_csv(itercsvname,
                                             skiprows=4)
            updatedDF = station_maxtemp_df.rename(columns={
                "Unnamed: 0": "Station Name",
                "deg" : "Latitude",
                "deg.1" : "Longitude",    # all dfs should have the same wonky
                "ft" : "Elevation",       # variable names.
                "Unnamed: 4" : "Year",
                "Unnamed: 5" : "Month",
                "Unnamed: 6" : "Day",
                "Degrees F" : "Max Temp",
                "Unnamed: 8" : "Max Temp Flag",
                "Degrees F.1" : "Normal Max Temp",
                "Degrees F.2" : "Departure from Normal Max Temp",
                "Unnamed: 11" : 'Departure from Normal Daily Maximum Air Temperature Flag'})
        
            outcleancsv = os.path.join(cleanmaxtemp,
                                       f'{stationnum}_maxclean.csv')
            updatedDF.to_csv(outcleancsv)

##### Doing the same as above for each station's 30-day MINIMUM temp:

In [60]:
for stationnum in just_stationnumbers:
    folderpath = os.path.join(stationsfolderpath,
                              stationnum)
    itercsvname = os.path.join(folderpath,
                               f'{stationnum}_mintemp.csv')
    with open(itercsvname, 'r',newline="") as itercsv:
        rowitems = []
        for row in itercsv:
            rowitems.append(row)
        if len(rowitems) < 6:             
            pass   
        else:
            station_mintemp_df = pd.read_csv(itercsvname,
                                             skiprows=4)
            updatedDF = station_mintemp_df.rename(columns={
                "Unnamed: 0": "Station Name",
                "deg" : "Latitude",
                "deg.1" : "Longitude",    
                "ft" : "Elevation",     
                "Unnamed: 4" : "Year",
                "Unnamed: 5" : "Month",
                "Unnamed: 6" : "Day",
                "Degrees F" : "Min Temp",
                "Unnamed: 8" : "Min Temp Flag",
                "Degrees F.1" : "Normal Min Temp",
                "Degrees F.2" : "Departure from Normal Min Temp",
                "Unnamed: 11" : 'Departure from Normal Daily Min Air Temperature Flag'})
        
            outcleancsv = os.path.join(cleanmintemp,
                                       f'{stationnum}_minclean.csv')
            updatedDF.to_csv(outcleancsv)

#### Example of what a 'clean' 30-day minimum temp CSV looks like for a given station:

In [61]:
updatedDF

,Station Name,Latitude,Longitude,Elevation,Year,Month,Day,Min Temp,Min Temp Flag,Normal Min Temp,Departure from Normal Min Temp,Departure from Normal Daily Min Air Temperature Flag
0,Zeeland,46.013378,-99.687587,2070,2021,3,22,12.920,NaN,21.0,-8.080,NaN
1,Zeeland,46.013378,-99.687587,2070,2021,3,23,24.978,NaN,21.0,3.978,NaN
2,Zeeland,46.013378,-99.687587,2070,2021,3,24,19.416,NaN,22.0,-2.584,NaN
3,Zeeland,46.013378,-99.687587,2070,2021,3,25,26.767,NaN,22.0,4.767,NaN
4,Zeeland,46.013378,-99.687587,2070,2021,3,26,20.849,NaN,22.0,-1.151,NaN
5,Zeeland,46.013378,-99.687587,2070,2021,3,27,14.828,NaN,23.0,-8.172,NaN
6,Zeeland,46.013378,-99.687587,2070,2021,3,28,7.160,NaN,23.0,-15.840,NaN
7,Zeeland,46.013378,-99.687587,2070,2021,3,29,22.498,NaN,24.0,-1.502,NaN
8,Zeeland,46.013378,-99.687587,2070,2021,3,30,11.120,NaN,24.0,-12.880,NaN
9,Zeeland,46.013378,-99.687587,2070,2021,3,31,6.008,NaN,24.0,-17.992,NaN


### O.) Using Pandas 'GroupBy' function to grab the maximum value among all 30 days of Max temp data for each; writing new CSVs to 'MaxTempGroupbyCSVs' folder:

In [62]:
for stationnum in just_stationnumbers:
    folderpath = os.path.join(stationsfolderpath,
                              stationnum)
    itercsvname = os.path.join(folderpath,
                               f'{stationnum}_maxtemp.csv')
    with open(itercsvname, 'r',newline="") as itercsv:
        rowitems = []
        for row in itercsv:
            rowitems.append(row)
        if len(rowitems) < 6:
            pass
        else:
            station_maxtemp_df = pd.read_csv(itercsvname,
                                             skiprows=4)
            updatedDF = station_maxtemp_df.rename(columns={
                "Unnamed: 0": "Station Name",
                "deg" : "Latitude",
                "deg.1" : "Longitude",
                "Degrees F" : "30 Day Max Temp"})
            outmaxtempcsv = os.path.join(wkdir,
                                         'MaxTempGroupbyCSVs',
                                         f'{stationnum}_maxtempgroupby.csv')
            
            groupbye = updatedDF.groupby(['Station Name',
                                         'Latitude',
                                         'Longitude']).agg({'30 Day Max Temp': ['max']}).reset_index()
                                            # grouping by MAX this time, to find the maximum
                                            # value between the 30 days..~
                                            # Grouping by max will again, not affect the lat and long
                                            # values, because they are all the same for each day.
            groupbye.columns = groupbye.columns.get_level_values(0)
            groupbye.to_csv(outmaxtempcsv)

### P.) Using Pandas 'GroupBy' function to grab the minimum value among all 30 days of Min temp data for each; writing new CSVs to 'MinTempGroupbyCSVs' folder:

In [63]:
for stationnum in just_stationnumbers:
    folderpath = os.path.join(stationsfolderpath,
                              stationnum)
    itercsvname = os.path.join(folderpath,
                               f'{stationnum}_mintemp.csv')
    with open(itercsvname, 'r',newline="") as itercsv:
        rowitems = []
        for row in itercsv:
            rowitems.append(row)
        if len(rowitems) < 6:
            pass
        else:
            station_mintemp_df = pd.read_csv(itercsvname,
                                             skiprows=4)
            updatedDF = station_mintemp_df.rename(columns={
                "Unnamed: 0": "Station Name",
                "deg" : "Latitude",
                "deg.1" : "Longitude",
                "Degrees F" : "30 Day Min Temp"}) # minimum this time..
            outmintempcsv = os.path.join(wkdir,
                                         'MinTempGroupbyCSVs',
                                         f'{stationnum}_mintempgroupby.csv')
            
            groupbye = updatedDF.groupby(['Station Name',
                                         'Latitude',
                                         'Longitude']).agg({'30 Day Min Temp': ['min']}).reset_index()
                                            # grouping by 'MIN' this time !!!!!~!~
            groupbye.columns = groupbye.columns.get_level_values(0)
            groupbye.to_csv(outmintempcsv)

### Q.) Concatenating all station MAX Temp Groupby CSVs into one CSV. This CSV contains each station name, its Latitude, Longitude, and maximum temp over 30 days:

In [64]:
maxtempGBconcat = os.path.join(maxtempgroupby,
                               'maxtempGBconcat.csv')

allFiles = glob.glob(maxtempgroupby + "/*.csv")
allFiles.sort()
with open(maxtempGBconcat, 'wb') as outfile:
    for i, fname in enumerate(allFiles):
        with open(fname, 'rb') as infile:
            if i != 0:
                infile.readline()  # Throw away header on all but first file
            # Block copy rest of file from input to output without parsing
            shutil.copyfileobj(infile, outfile)
            print(fname + " has been imported.")

C:\\Users\\michaelfelzan\\Documents\\arc ii lab IV\2021-04-21_NDAWN\MaxTempGroupbyCSVs\100_maxtempgroupby.csv has been imported.
C:\\Users\\michaelfelzan\\Documents\\arc ii lab IV\2021-04-21_NDAWN\MaxTempGroupbyCSVs\101_maxtempgroupby.csv has been imported.
C:\\Users\\michaelfelzan\\Documents\\arc ii lab IV\2021-04-21_NDAWN\MaxTempGroupbyCSVs\102_maxtempgroupby.csv has been imported.
C:\\Users\\michaelfelzan\\Documents\\arc ii lab IV\2021-04-21_NDAWN\MaxTempGroupbyCSVs\103_maxtempgroupby.csv has been imported.
C:\\Users\\michaelfelzan\\Documents\\arc ii lab IV\2021-04-21_NDAWN\MaxTempGroupbyCSVs\104_maxtempgroupby.csv has been imported.
C:\\Users\\michaelfelzan\\Documents\\arc ii lab IV\2021-04-21_NDAWN\MaxTempGroupbyCSVs\105_maxtempgroupby.csv has been imported.
C:\\Users\\michaelfelzan\\Documents\\arc ii lab IV\2021-04-21_NDAWN\MaxTempGroupbyCSVs\106_maxtempgroupby.csv has been imported.
C:\\Users\\michaelfelzan\\Documents\\arc ii lab IV\2021-04-21_NDAWN\MaxTempGroupbyCSVs\107_maxtem

C:\\Users\\michaelfelzan\\Documents\\arc ii lab IV\2021-04-21_NDAWN\MaxTempGroupbyCSVs\66_maxtempgroupby.csv has been imported.
C:\\Users\\michaelfelzan\\Documents\\arc ii lab IV\2021-04-21_NDAWN\MaxTempGroupbyCSVs\67_maxtempgroupby.csv has been imported.
C:\\Users\\michaelfelzan\\Documents\\arc ii lab IV\2021-04-21_NDAWN\MaxTempGroupbyCSVs\68_maxtempgroupby.csv has been imported.
C:\\Users\\michaelfelzan\\Documents\\arc ii lab IV\2021-04-21_NDAWN\MaxTempGroupbyCSVs\69_maxtempgroupby.csv has been imported.
C:\\Users\\michaelfelzan\\Documents\\arc ii lab IV\2021-04-21_NDAWN\MaxTempGroupbyCSVs\6_maxtempgroupby.csv has been imported.
C:\\Users\\michaelfelzan\\Documents\\arc ii lab IV\2021-04-21_NDAWN\MaxTempGroupbyCSVs\70_maxtempgroupby.csv has been imported.
C:\\Users\\michaelfelzan\\Documents\\arc ii lab IV\2021-04-21_NDAWN\MaxTempGroupbyCSVs\71_maxtempgroupby.csv has been imported.
C:\\Users\\michaelfelzan\\Documents\\arc ii lab IV\2021-04-21_NDAWN\MaxTempGroupbyCSVs\72_maxtempgroupby.

### R.) Concatenating all station MIN Temp Groupby CSVs into one CSV. This CSV contains each station name, its Latitude, Longitude, and minimum temp over 30 days:

In [65]:
mintempGBconcat = os.path.join(mintempgroupby,
                               'mintempGBconcat.csv')

allFiles = glob.glob(mintempgroupby + "/*.csv")
allFiles.sort()
with open(mintempGBconcat, 'wb') as outfile:
    for i, fname in enumerate(allFiles):
        with open(fname, 'rb') as infile:
            if i != 0:
                infile.readline()  # Throw away header on all but first file
            # Block copy rest of file from input to output without parsing
            shutil.copyfileobj(infile, outfile)
            print(fname + " has been imported.")

C:\\Users\\michaelfelzan\\Documents\\arc ii lab IV\2021-04-21_NDAWN\MinTempGroupbyCSVs\100_mintempgroupby.csv has been imported.
C:\\Users\\michaelfelzan\\Documents\\arc ii lab IV\2021-04-21_NDAWN\MinTempGroupbyCSVs\101_mintempgroupby.csv has been imported.
C:\\Users\\michaelfelzan\\Documents\\arc ii lab IV\2021-04-21_NDAWN\MinTempGroupbyCSVs\102_mintempgroupby.csv has been imported.
C:\\Users\\michaelfelzan\\Documents\\arc ii lab IV\2021-04-21_NDAWN\MinTempGroupbyCSVs\103_mintempgroupby.csv has been imported.
C:\\Users\\michaelfelzan\\Documents\\arc ii lab IV\2021-04-21_NDAWN\MinTempGroupbyCSVs\104_mintempgroupby.csv has been imported.
C:\\Users\\michaelfelzan\\Documents\\arc ii lab IV\2021-04-21_NDAWN\MinTempGroupbyCSVs\105_mintempgroupby.csv has been imported.
C:\\Users\\michaelfelzan\\Documents\\arc ii lab IV\2021-04-21_NDAWN\MinTempGroupbyCSVs\106_mintempgroupby.csv has been imported.
C:\\Users\\michaelfelzan\\Documents\\arc ii lab IV\2021-04-21_NDAWN\MinTempGroupbyCSVs\107_mintem

C:\\Users\\michaelfelzan\\Documents\\arc ii lab IV\2021-04-21_NDAWN\MinTempGroupbyCSVs\63_mintempgroupby.csv has been imported.
C:\\Users\\michaelfelzan\\Documents\\arc ii lab IV\2021-04-21_NDAWN\MinTempGroupbyCSVs\64_mintempgroupby.csv has been imported.
C:\\Users\\michaelfelzan\\Documents\\arc ii lab IV\2021-04-21_NDAWN\MinTempGroupbyCSVs\65_mintempgroupby.csv has been imported.
C:\\Users\\michaelfelzan\\Documents\\arc ii lab IV\2021-04-21_NDAWN\MinTempGroupbyCSVs\66_mintempgroupby.csv has been imported.
C:\\Users\\michaelfelzan\\Documents\\arc ii lab IV\2021-04-21_NDAWN\MinTempGroupbyCSVs\67_mintempgroupby.csv has been imported.
C:\\Users\\michaelfelzan\\Documents\\arc ii lab IV\2021-04-21_NDAWN\MinTempGroupbyCSVs\68_mintempgroupby.csv has been imported.
C:\\Users\\michaelfelzan\\Documents\\arc ii lab IV\2021-04-21_NDAWN\MinTempGroupbyCSVs\69_mintempgroupby.csv has been imported.
C:\\Users\\michaelfelzan\\Documents\\arc ii lab IV\2021-04-21_NDAWN\MinTempGroupbyCSVs\6_mintempgroupby.

# 5.) USING ARCPY TO CREATE POINT FEATURE CLASSES OUT OF CONCATENATED CSV FILES

### S.) Setting up directory to store Arcpy-related files:

In [67]:
arcpro_folder = os.path.join(wkdir,'ARCPRO')
os.mkdir(arcpro_folder)

In [70]:
arcpy.env.workspace = wkdir

##### String which accesses WGS84 GCS when inputted into the CRS param of Arcpy functions:

In [71]:
wgs84_GCS = "GEOGCS['GCS_WGS_1984',DATUM['D_WGS_1984',SPHEROID['WGS_1984',6378137.0,298.257223563]],PRIMEM['Greenwich',0.0],UNIT['Degree',0.0174532925199433]];-400 -400 1000000000;-100000 10000;-100000 10000;8.98315284119521E-09;0.001;0.001;IsHighPrecision"

## T.) Using the Arcpy Management "XYTableToPoint" function to convert the 'Ave Temp Groupby Concatenation' CSV into a point feature class, using the Longitude and Latitude fields for the X and Y:

In [74]:
avetempstations = os.path.join(arcpro_folder, "weatherstations_avetemp")

arcpy.management.XYTableToPoint(avetempGBconcat,
                                avetempstations,
                                "Longitude",
                                "Latitude", 
                             None,
                                wgs84_GCS)

<Result 'C:\\\\Users\\\\michaelfelzan\\\\Documents\\\\arc ii lab IV\\2021-04-21_NDAWN\\ARCPRO\\weatherstations_avetemp.shp'>

## ~ ~ ~ ~ ~ ~ ~

### ~ ~ ~ ~ ~ ~

##### Instead of joining the Max Temp and Min Temp tables to the point feature class created in the above step, I decided , because those tables `also` contain Lat and Long fields, it would be less steps to just create two more point feature classes -- one for all stations' max temps over 30 days, and another for all stations' min temp over 30 days.

### ~ ~ ~ ~ ~ .,

### U.) Using the Arcpy Management "XYTableToPoint" function to convert the 'Max Temp Groupby Concatenation' CSV into its own point feature class:

In [76]:
maxtempstations = os.path.join(arcpro_folder, "weather_stations_maxtemp")

arcpy.management.XYTableToPoint(maxtempGBconcat,
                                maxtempstations,
                                "Longitude",
                                "Latitude", 
                                None,
                                wgs84_GCS)

<Result 'C:\\\\Users\\\\michaelfelzan\\\\Documents\\\\arc ii lab IV\\2021-04-21_NDAWN\\ARCPRO\\weather_stations_maxtemp.shp'>

### V.) Using the Arcpy Management "XYTableToPoint" function to convert the 'Min Temp Groupby Concatenation' CSV into its own point feature class:

In [77]:
mintempstations = os.path.join(arcpro_folder, "weather_stations_mintemp")

arcpy.management.XYTableToPoint(mintempGBconcat,
                                mintempstations,
                                "Longitude",
                                "Latitude", 
                                None,
                                wgs84_GCS)

<Result 'C:\\\\Users\\\\michaelfelzan\\\\Documents\\\\arc ii lab IV\\2021-04-21_NDAWN\\ARCPRO\\weather_stations_mintemp.shp'>

##### designating path to weatherstations_avetemp.shp:

In [81]:
avetempstations_SHP = os.path.join(arcpro_folder, "weatherstations_avetemp.shp")

### W.) Creating file geodatabase in 'ARCPRO' folder ('grid' files, which the Kriging function will output in the next step, have(?) to be stored in esri filegeodatabases:

In [83]:
arcpy.CreateFileGDB_management(arcpro_folder, "Lab4_fGDB.gdb")

<Result 'C:\\\\Users\\\\michaelfelzan\\\\Documents\\\\arc ii lab IV\\2021-04-21_NDAWN\\ARCPRO\\Lab4_fGDB.gdb'>

In [84]:
lab4GBD = os.path.join(arcpro_folder, "Lab4_fGDB.gdb" )

# 6.) USING ARCPY TO SPATIALLY INTERPOLATE MONTHLY AVE, MAX, AND MIN TEMPERATURE VALUES

## X.) Running arcpy.sa.Kriging() function on Ave Temp weather stations FC, outputting interpolated surface map of average monthly temperature:

In [86]:
# Kriging(in_point_features, 
#         z_field, 
#        semiVariogram_props, 
#        {cell_size}, 
#        {search_radius}, 
#        {out_variance_prediction_raster})


avetemp_OrdKrig = os.path.join(lab4GBD, "avetemp_OrdKrig")

out_surface_raster = arcpy.sa.Kriging(avetempstations_SHP,
                                      "F30_Day_Av",
                                      "Spherical 0.014478 # # #",
                                      0.014477764,
                                      "VARIABLE 12",
                                      None)
                  
out_surface_raster.save(avetemp_OrdKrig)

## Y.) Running arcpy.sa.Kriging() function on Max Temp weather stations FC, outputting interpolated surface map of monthly max temperature:

In [88]:
maxtempstations_SHP = os.path.join(arcpro_folder, "weather_stations_maxtemp.shp")
maxtemp_OrdKrig = os.path.join(lab4GBD, "maxtemp_OrdKrig")

out_surface_raster = arcpy.sa.Kriging(maxtempstations_SHP,
                                      "F30_Day_Ma",
                                      "Spherical 0.014478 # # #",
                                      0.014477764,
                                      "VARIABLE 12",
                                      None)
                  
out_surface_raster.save(maxtemp_OrdKrig)

## Z.) Running arcpy.sa.Kriging() function on Min Temp weather stations FC, outputting interpolated surface map of monthly minimum temperature:

In [89]:
mintempstations_SHP = os.path.join(arcpro_folder, "weather_stations_mintemp.shp")
mintemp_OrdKrig = os.path.join(lab4GBD, "mintemp_OrdKrig")

out_surface_raster = arcpy.sa.Kriging(mintempstations_SHP,
                                      "F30_Day_Mi",
                                      "Spherical 0.014478 # # #",
                                      0.014477764,
                                      "VARIABLE 12",
                                      None)
                  
out_surface_raster.save(mintemp_OrdKrig)